# Constants

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
deployment_name =  "gpt-4o-mini" # gpt-4o-mini or gpt-4o

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

# Just for testing: List current AI Foundry Agents

In [3]:
project_client.agents.list_agents()

{'object': 'list', 'data': [], 'first_id': None, 'last_id': None, 'has_more': False}

# Create AI Foundry Agent with its CodeInterpreterTool for AI Agents

In [4]:
# First, create the Code Interpreter Tool for AI Foundry Agents...
from azure.ai.projects.models import CodeInterpreterTool
code_interpreter = CodeInterpreterTool()
print(f"Code interpreter definitions: {code_interpreter.definitions}")
print(f"Code interpreter resources: {code_interpreter.resources}")

# ...then, create the AI Foundry Agent, attaching CodeInterpreterTool object
ai_agent = project_client.agents.create_agent(
    model=deployment_name,
    name="my-agent",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources
)

ai_agent.items

Code interpreter definitions: [{'type': 'code_interpreter'}]
Code interpreter resources: {}


<bound method _MyMutableMapping.items of {'id': 'asst_UBeJjrKy0on9BrKZSOdYUCB9', 'object': 'assistant', 'created_at': 1737212838, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}>

# Just for testing: create a new agent and delete it

In [5]:
# Create a second agent
ai_agent_02 = project_client.agents.create_agent(
    model=deployment_name,
    name="my-agent",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)

# List all agents
print(f"All agents, including the second one: {project_client.agents.list_agents()['data']}\n")

project_client.agents.delete_agent(ai_agent_02.id)

# List all agents
print(f"All agents, after deleting the second one: {project_client.agents.list_agents()['data']}")

All agents, including the second one: [{'id': 'asst_gBR402GwLLezaOGLpdWmwt6L', 'object': 'assistant', 'created_at': 1737212839, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}, {'id': 'asst_UBeJjrKy0on9BrKZSOdYUCB9', 'object': 'assistant', 'created_at': 1737212838, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}]

All agents, after deleting the second one: [{'id': 'asst_UBeJjrKy0on9BrKZSOdYUCB9', 'object': 'assistant', 'created_at': 1737212838, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions'

# Create the thread and attach a new message to it

In [6]:
# Create a thread
thread = project_client.agents.create_thread()

print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
    Could you please create a bar chart for the operating profit using 
    the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, 
    Company D: $1.8 million
    """,
)
print(f"Messages: {project_client.agents.list_messages(thread_id=thread.id)}")

Created thread: {'id': 'thread_WYNpD4pZsbXRiXXfe0lfPfad', 'object': 'thread', 'created_at': 1737212841, 'metadata': {}, 'tool_resources': {}}

Messages: {'object': 'list', 'data': [{'id': 'msg_LG9i4vtykyYyQ5qIBbNIRjI7', 'object': 'thread.message', 'created_at': 1737212841, 'assistant_id': None, 'thread_id': 'thread_WYNpD4pZsbXRiXXfe0lfPfad', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': '\n    Could you please create a bar chart for the operating profit using \n    the following data and provide the file to me? \n    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, \n    Company D: $1.8 million\n    ', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_LG9i4vtykyYyQ5qIBbNIRjI7', 'last_id': 'msg_LG9i4vtykyYyQ5qIBbNIRjI7', 'has_more': False}


# Run the agent syncrhonously

In [7]:
%%time

# Run the agent
run = project_client.agents.create_and_process_run\
    (thread_id=thread.id, assistant_id=ai_agent.id,)

print(f"Run status: {run.status}.\nRun details: {run}")

Run status: RunStatus.COMPLETED.
Run details: {'id': 'run_PFlwiHrpoDdS4Zf8Rxpk4Dqt', 'object': 'thread.run', 'created_at': 1737212842, 'assistant_id': 'asst_UBeJjrKy0on9BrKZSOdYUCB9', 'thread_id': 'thread_WYNpD4pZsbXRiXXfe0lfPfad', 'status': 'completed', 'started_at': 1737212843, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1737212853, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 406, 'completion_tokens': 218, 'total_tokens': 624}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}
CPU times: total: 31.2 ms
Wall time: 11.1 s


# Fetch messages from the thread after the agent run execution

In [8]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Here are the {len(messages.data)} messages, starting with the most recent one:\n")
    i=0
    for m in messages.data:
        j = 0
        i += 1
        print(f"\n===== MESSAGE {i} =====")
        for c in m.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages, starting with the most recent one:


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: I have created the bar chart for the operating profit. You can download it using the link below:

[Download Operating Profit Chart](sandbox:/mnt/data/operating_profit_chart.png)
>>> Annotation in MessageTextContent 1 of message 1: sandbox:/mnt/data/operating_profit_chart.png


===== MESSAGE 2 =====

CONTENT 1 (MessageTextContent) --> Text: 
    Could you please create a bar chart for the operating profit using 
    the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, 
    Company D: $1.8 million
    


# Retrieve and download eventual annotations

In [9]:
# messages.file_path_annotations[0].text.split('/')[-1]
print (f"Nr. of file path annotations: {len(messages.file_path_annotations)}\n")

i=0
for file_path_annotation in messages.file_path_annotations:
    i += 1
    print(f"{i} - File annotation paths: {file_path_annotation}")
    file_name = file_path_annotation.text.split('/')[-1]
    project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annoation {i} saved as file to: {os.getcwd()}\\{file_name}")

Nr. of file path annotations: 1

1 - File annotation paths: {'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_chart.png', 'start_index': 132, 'end_index': 176, 'file_path': {'file_id': 'assistant-NJHby1rOpHsLONikWXKT48GN'}}
File annoation 1 saved as file to: E:\Users\mauromi\source\git_repos\aaas\operating_profit_chart.png


# Retrieve and download eventual images

In [10]:
print (f"Nr. of image contents: {len(messages.image_contents)}\n")

i=0
# Generate an image file for the bar chart
for image_content in messages.image_contents:
    i += 1
    print(f"{i} - Image content: {image_content}")
    file_name = f"{image_content.image_file.file_id}_image_content.png"
    project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
    print(f"Image content {i} file to: {os.getcwd()}\\{file_name}")

Nr. of image contents: 0



# Delete all agents

In [11]:
print (f"Nr. of existing agents that will be deleted: {len(project_client.agents.list_agents()['data'])}\n")

i = 0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"{i} - Agent {pca.name} ({pca.id}) is not deleted.")

Nr. of existing agents that will be deleted: 1

1 - Agent my-agent (asst_UBeJjrKy0on9BrKZSOdYUCB9) is not deleted.


# HIC SUNT LEONES